In [ ]:
!pip install scikit-surprise





  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-win_amd64.whl size=1298937 sha256=ccb8f0b0c6cea65cac0d2a641d0afc5d769279467177a20a20b81ab39109c374
  Stored in directory: c:\users\gshas\appdata\local\pip\cache\wheels\2a\8f\6e\7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy
import pandas as pd

# Load built-in MovieLens 100k dataset
data = Dataset.load_builtin('ml-100k')


Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 c-


Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 y


Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\gshas/.surprise_data/ml-100k


In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


In [ ]:
model = SVD()
model.fit(trainset)


In [ ]:
predictions = model.test(testset)

# Calculate RMSE
rmse = accuracy.rmse(predictions)


RMSE: 0.9349


In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Sort and get top N
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n(predictions, n=5)

# Show recommendations for user '196'
print("Top 5 recommendations for user 196:")
for movie_id, rating in top_n['196']:
    print(f"Movie ID: {movie_id}, Predicted Rating: {rating:.2f}")


Top 5 recommendations for user 196:
Movie ID: 286, Predicted Rating: 4.19
Movie ID: 173, Predicted Rating: 4.01
Movie ID: 306, Predicted Rating: 3.90
Movie ID: 116, Predicted Rating: 3.79
Movie ID: 153, Predicted Rating: 3.68


In [ ]:
# 5-fold cross-validation
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9384  0.9326  0.9332  0.9399  0.9348  0.9358  0.0029  
MAE (testset)     0.7401  0.7347  0.7360  0.7394  0.7364  0.7373  0.0021  
Fit time          1.14    0.99    0.84    0.97    1.06    1.00    0.10    
Test time         0.25    0.12    0.19    0.13    0.21    0.18    0.05    


{'test_rmse': array([0.93840515, 0.93264859, 0.93318339, 0.93985932, 0.93483339]),
 'test_mae': array([0.74005225, 0.73465187, 0.73600375, 0.73943196, 0.73638222]),
 'fit_time': (1.1417016983032227,
  0.9899771213531494,
  0.8412485122680664,
  0.9729495048522949,
  1.0627598762512207),
 'test_time': (0.24984431266784668,
  0.11957240104675293,
  0.1898207664489746,
  0.1253366470336914,
  0.21424126625061035)}